In [ ]:
import pandas as pd
import heapq
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import itertools

In [ ]:
df = pd.read_csv('metro_cdmx_aristas.csv')
df['from_stop_name'] = df['from_stop_name'].str.strip()
df['to_stop_name'] = df['to_stop_name'].str.strip()


graph = {}

for _, row in df.iterrows():
    u = row['from_stop_name']
    v = row['to_stop_name']
    weight = row['mean_travel_time_min']

    line_raw = str(row['route_short_name']).strip()
    line = line_raw.lstrip('L')

    if u not in graph: graph[u] = []
    graph[u].append((v, weight, line))
    
all_stations = sorted(list(graph.keys()))

def calcular_ruta(origen, destino):
    # Creamos un generador de números únicos para desempatar
    counter = itertools.count() 
    
    # Estructura de la tupla en la cola: 
    # (Costo, Contador_Unico, Nodo_Actual, Linea_Actual, Camino)
    pq = [(0, next(counter), origen, None, [])]
    
    min_costs = {}
    
    while pq:
        # Desempaquetamos incluyendo el contador (que ignoramos con _)
        current_time, _, current_node, current_line, path = heapq.heappop(pq)

        # Si llegamos al destino, retornamos el resultado
        if current_node == destino:
            return current_time, path

        # Optimización: Si ya encontramos un camino más rápido a este estado
        state = (current_node, current_line)
        if state in min_costs and min_costs[state] <= current_time:
            continue
        min_costs[state] = current_time

        # Explorar vecinos
        if current_node in graph:
            for neighbor, travel_time, next_line in graph[current_node]:
                new_time = current_time + travel_time
                penalty = 0
                action = f"Viajar a {neighbor}"

                # Lógica de penalización por transbordo
                if current_line is not None and current_line != next_line:
                    penalty = 4 # 4 minutos por cambio de línea
                    action = f"Transbordo (baja L{current_line}, sube L{next_line}) en dirección {neighbor}"

                total_cost = new_time + penalty

                # Construir nuevo camino
                new_path = path + [{
                    'station': neighbor,
                    'line': next_line,
                    'action': action,
                    'time_leg': travel_time,
                    'penalty': penalty
                }]

                # Agregamos next(counter) en la segunda posición para romper empates
                heapq.heappush(pq, (total_cost, next(counter), neighbor, next_line, new_path))

    return float('inf'), []

In [ ]:
style = {'description_width': 'initial'}

w_origen = widgets.Dropdown(options=all_stations, description='Estación Origen:', style=style)
w_destino = widgets.Dropdown(options=all_stations, description='Estación Destino:', style=style)
w_boton = widgets.Button(description="Calcular Ruta", button_style='success')
w_output = widgets.Output()

def on_button_click(b):
    with w_output:
        clear_output()
        start = w_origen.value
        end = w_destino.value

        if start == end:
            print("La estación de origen y destino son la misma.")
            return

        tiempo_total, pasos = calcular_ruta(start, end)

        if tiempo_total == float('inf'):
            print(f"No se encontró una ruta entre {start} y {end}.")
        else:
            print(f"RUTA CALCULADA")
            print(f"Tiempo total estimado: {tiempo_total:.2f} minutos")
            print("-" * 60)
            print(f"INICIO: {start}")

            current_line = pasos[0]['line']
            print(f"  Ingresa a la Línea {current_line}")

            for paso in pasos:
                tiempo_leg = paso['time_leg']
                extra = ""
                if paso['penalty'] > 0:
                    extra = f"4 min transbordo"
                    print(f"CAMBIO DE LÍNEA: {paso['action'], extra}")
                else:
                    print(f"   • {paso['station']} ({tiempo_leg:.1f} min)")

            print(f"LLEGADA: {end}")
w_boton.on_click(on_button_click)

display(widgets.VBox([
    widgets.HTML("<h2>Planeador de Rutas Metro CDMX</h2>"),
    widgets.HBox([w_origen, w_destino]),
    w_boton,
    w_output
]))
